## **Menginstall Library**

In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
!pip install roboflow
!pip install ultralytics

from roboflow import Roboflow

## **Mendowload dataset yang sebelumnya di buat di roboflow**

In [ ]:
rf = Roboflow(api_key=API_KEY)
project = rf.workspace("novilm").project("frog_dataset-7aual")
version = project.version(1)
dataset = version.download("yolov8")

# Simpan dataset di folder "datasets/frog"
dataset = version.download("yolov8", location="/kaggle/working/datasets/frog")

## **Mengedit data.yaml**

kode ini di jalankan di `kaggle` sehingga untuk mengedit kodenya menggunakan kode berikut. `data.yaml` harus di edit terlebih dahulu untuk memetakan lokasi data `train` `test` dan `val` pada model YOLOv8.

`!cat` di gunakan untuk menampilkan isi file, isi file ini kemudian di copy

In [ ]:
!cat /kaggle/working/datasets/frog/data.yaml

`%%writefile` digunakan untuk menulis ulang isi file berdasarkan isi fle yang sudah di copy sebelumnya

In [ ]:
%%writefile /kaggle/working/datasets/frog/data.yaml
names:
- frog
nc: 1
roboflow:
  license: CC BY 4.0
  project: frog_dataset-7aual
  url: https://universe.roboflow.com/novilm/frog_dataset-7aual/dataset/1
  version: 1
  workspace: novilm
test: /kaggle/working/datasets/frog/test
train: /kaggle/working/datasets/frog/train
val: /kaggle/working/datasets/frog/valid

setelah file selesa di edit selanjutnya filenya di cek lagi

In [ ]:
!cat /kaggle/working/datasets/frog/data.yaml

## **Training YOLOv8**

In [ ]:
!yolo task=detect mode=train model=yolov8s.pt data=/kaggle/working/datasets/frog/data.yaml epochs=25 imgsz=800 plots=True

setelah training selesai di lakukan, akan muncul folder baru bernama `train` yang di dalamnya akan ada gambar-gambar plot selama pelatihan dan akan ada gambar selama proses test  dan validasi. kok bisa ada gambar? hal ini di karenakan pada sintaks sebelumnya `plots=True` jika di set false maka tidak akan di simpan hasil validasi dan plot-plotnya

In [ ]:

# Tentukan path ke direktori gambar
image_path = "/kaggle/working/runs/detect/train"

# Daftar ekstensi file gambar yang diterima
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']

# Ambil semua nama file di dalam image_path yang memiliki ekstensi gambar
fnames = [image_path + '/' + fname for fname in os.listdir(image_path) 
          if any(fname.lower().endswith(ext) for ext in image_extensions)]

# Cek jika ada file gambar
if fnames:
    # Loop melalui semua file gambar
    for fname in fnames:
        try:
            # Buka gambar dan tampilkan
            im = Image.open(fname)
            plt.imshow(im)
            plt.title(f"Gambar: {fname.split('/')[-1]}")
            plt.axis('off')  # Menyembunyikan axis
            plt.show()
        except Exception as e:
            print(f"Terjadi kesalahan saat membuka gambar {fname}: {e}")
else:
    print("Tidak ada gambar di direktori.")


## **Inference Model**

Sekilas Sintaksnya sama seperti sebelumnya, bedanya di sini modelnua menggunakan model hasil train sebelumnya pada folde `train`

In [ ]:
!yolo task=detect mode=predict model=/kaggle/working/runs/detect/train/weights/best.pt conf=0.25 source=/kaggle/input/test-video/3293556-hd_1080_1920_30fps.mp4 save=True